<a href="https://colab.research.google.com/github/LilyMimosa/Capstone-Project/blob/main/Salinan_dari_dicoding_capstone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **I. IMPORT**

In [ ]:
import kagglehub
import os
from google.colab import drive

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer, IterativeImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from imblearn.pipeline import Pipeline as ImbPipeline
from imblearn.over_sampling import SMOTE
from sklearn.metrics import roc_auc_score

import joblib

pd.set_option('display.max_columns', None)

# **II. FILE PATH**

In [ ]:
KAGGLE_DATASET_REPO = "kamilpytlak/personal-key-indicators-of-heart-disease"
KAGGLE_FILE_IN_REPO = "2022/heart_2022_with_nans.csv"

GDRIVE_PROJECT_DIR = '/content/drive/MyDrive/dicoding_capstone'

# DATASETS
GDRIVE_DATASETS_DIR = 'datasets'
GDRIVE_DATASETS_PATH = os.path.join(GDRIVE_PROJECT_DIR, GDRIVE_DATASETS_DIR)

# MODELS
GDRIVE_MODELS_DIR = 'models'
GDRIVE_MODEL_PATH = os.path.join(GDRIVE_PROJECT_DIR, GDRIVE_MODELS_DIR)

# Original dataset
GDRIVE_ORIGINAL_DATASET_FILE = '1.original.csv'
GDRIVE_ORIGINAL_FILE_PATH = os.path.join(GDRIVE_DATASETS_PATH, GDRIVE_ORIGINAL_DATASET_FILE)

# No duplicate dataset
GDRIVE_NO_DUPLICATE_DATASET_FILE = '2.no_duplicate.csv'
GDRIVE_NO_DUPLICATE_DATASET_PATH = os.path.join(GDRIVE_DATASETS_PATH, GDRIVE_NO_DUPLICATE_DATASET_FILE)

# No irrelevant
GDRIVE_NO_IRRELEVANT_DATASET_FILE = '3.no_irrelevant.csv'
GDRIVE_NO_IRRELEVANT_DATASET_PATH = os.path.join(GDRIVE_DATASETS_PATH, GDRIVE_NO_IRRELEVANT_DATASET_FILE)

# Fixed binary
GDRIVE_FIXED_BINARY_DATASET_FILE = '4.fixed_binary.csv'
GDRIVE_FIXED_BINARY_DATASET_PATH = os.path.join(GDRIVE_DATASETS_PATH, GDRIVE_FIXED_BINARY_DATASET_FILE)

# Fixed ordinal
GDRIVE_FIXED_ORDINAL_DATASET_FILE = '5.fixed_ordinal.csv'
GDRIVE_FIXED_ORDINAL_DATASET_PATH = os.path.join(GDRIVE_DATASETS_PATH, GDRIVE_FIXED_ORDINAL_DATASET_FILE)

# Fixed nominal
GDRIVE_FIXED_NOMINAL_DATASET_FILE = '6.fixed_nominal.csv'
GDRIVE_FIXED_NOMINAL_DATASET_PATH = os.path.join(GDRIVE_DATASETS_PATH, GDRIVE_FIXED_NOMINAL_DATASET_FILE)

# With target
GDRIVE_WITH_TARGET_DATASET_FILE = '7.with_target.csv'
GDRIVE_WITH_TARGET_FILE_PATH = os.path.join(GDRIVE_DATASETS_PATH, GDRIVE_WITH_TARGET_DATASET_FILE)

# Lowercase object
GDRIVE_LOWERCASE_OBJECT_DATASET_FILE = '8.lowercase_object.csv'
GDRIVE_LOWERCASE_OBJECT_DATASET_PATH = os.path.join(GDRIVE_DATASETS_PATH, GDRIVE_LOWERCASE_OBJECT_DATASET_FILE)

# Base preprocessed
GDRIVE_BASE_PREPROCESSED_DATASET_FILE = '9.base_preprocessed.csv'
GDRIVE_BASE_PREPROCESSED_DATASET_PATH = os.path.join(GDRIVE_DATASETS_PATH, GDRIVE_BASE_PREPROCESSED_DATASET_FILE)

# Distance based scaled
GDRIVE_DISTANCE_BASE_SCALED_DATASET_FILE = '10.distance_base_scaled.csv'
GDRIVE_DISTANCE_BASE_SCALED_DATASET_PATH = os.path.join(GDRIVE_DATASETS_PATH, GDRIVE_DISTANCE_BASE_SCALED_DATASET_FILE)

# Tree based scaled
GDRIVE_TREE_BASE_SCALED_DATASET_FILE = '11.tree_base_scaled.csv'
GDRIVE_TREE_BASE_SCALED_DATASET_PATH = os.path.join(GDRIVE_DATASETS_PATH, GDRIVE_TREE_BASE_SCALED_DATASET_FILE)

# **III. REUSABLE FUNCTIONS**

**Dataframe summarization**

In [ ]:
def summarize_dataframe(df: pd.DataFrame):
    summary = pd.DataFrame({
        "Non-Null Count": df.notnull().sum(),
        "Null Count": df.isnull().sum(),
        "Dtype": df.dtypes,
        "Unique Values": df.nunique()
    })

    print(f"Dataset shape:")
    print(f"Rows: {df.shape[0]}, Columns: {df.shape[1]}")

    print("\nSummary:")
    print(summary)

    print("\nType Count:")
    print(df.dtypes.value_counts())

    return summary

**Dataframe unique values**

In [ ]:
def show_unique_values(df, max_display=40):
    for col in df.columns:
        uniques = df[col].unique()[:max_display]
        print(f"\nColumn: {col}")
        print(f"Unique values (max {max_display} shown): {uniques}")
        print(f"Total unique values: {df[col].nunique()}")

# **IV. DOWNLOAD ORIGINAL DATASET**

In [ ]:
drive.mount('/content/drive')

os.makedirs(GDRIVE_PROJECT_DIR, exist_ok=True)

if os.path.exists(GDRIVE_ORIGINAL_FILE_PATH):
    print(f"Original dataset found in Google Drive: {GDRIVE_ORIGINAL_FILE_PATH}")
    df = pd.read_csv(GDRIVE_ORIGINAL_FILE_PATH)
else:
    print("Downloading dataset from Kaggle")
    df = kagglehub.dataset_load(
      kagglehub.KaggleDatasetAdapter.PANDAS,
      KAGGLE_DATASET_REPO,
      KAGGLE_FILE_IN_REPO,
    )

    print(f"Saving to Google Drive: {GDRIVE_ORIGINAL_FILE_PATH}")
    df.to_csv(GDRIVE_ORIGINAL_FILE_PATH, index=False)

df.sample(n=10)

Mounted at /content/drive
Original dataset found in Google Drive: /content/drive/MyDrive/dicoding_capstone/datasets/1.original.csv


,State,Sex,GeneralHealth,PhysicalHealthDays,MentalHealthDays,LastCheckupTime,PhysicalActivities,SleepHours,RemovedTeeth,HadHeartAttack,HadAngina,HadStroke,HadAsthma,HadSkinCancer,HadCOPD,HadDepressiveDisorder,HadKidneyDisease,HadArthritis,HadDiabetes,DeafOrHardOfHearing,BlindOrVisionDifficulty,DifficultyConcentrating,DifficultyWalking,DifficultyDressingBathing,DifficultyErrands,SmokerStatus,ECigaretteUsage,ChestScan,RaceEthnicityCategory,AgeCategory,HeightInMeters,WeightInKilograms,BMI,AlcoholDrinkers,HIVTesting,FluVaxLast12,PneumoVaxEver,TetanusLast10Tdap,HighRiskLastYear,CovidPos
81005,Georgia,Female,Good,0.0,0.0,Within past 2 years (1 year but less than 2 ye...,Yes,7.0,None of them,No,No,No,No,No,No,No,No,Yes,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"White only, Non-Hispanic",Age 70 to 74,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29606,California,Female,Excellent,0.0,0.0,Within past 2 years (1 year but less than 2 ye...,No,7.0,"6 or more, but not all",No,No,No,No,No,No,No,No,Yes,Yes,No,No,No,No,No,No,Former smoker,Never used e-cigarettes in my entire life,Yes,"Other race only, Non-Hispanic",Age 75 to 79,1.68,90.72,32.28,No,No,No,Yes,"Yes, received tetanus shot but not sure what type",No,No
442319,Puerto Rico,Female,Good,0.0,0.0,Within past year (anytime less than 12 months ...,No,6.0,"6 or more, but not all",No,No,No,No,No,No,No,No,Yes,No,No,No,No,No,No,No,Never smoked,Not at all (right now),Yes,Hispanic,Age 75 to 79,1.60,89.81,35.07,No,No,No,Yes,"No, did not receive any tetanus shot in the pa...",No,Yes
247786,New Jersey,Male,Very good,0.0,5.0,Within past year (anytime less than 12 months ...,Yes,7.0,None of them,No,No,No,Yes,No,No,No,No,Yes,No,No,No,No,No,No,No,Never smoked,Never used e-cigarettes in my entire life,No,"White only, Non-Hispanic",Age 40 to 44,1.85,83.91,24.41,Yes,Yes,Yes,No,"Yes, received Tdap",No,Yes
236715,Nevada,Female,Very good,0.0,0.0,Within past 5 years (2 years but less than 5 y...,Yes,8.0,None of them,No,No,No,No,No,No,No,No,Yes,No,No,No,No,No,No,No,Never smoked,Not at all (right now),No,Hispanic,Age 25 to 29,1.62,62.00,23.62,Yes,Yes,No,No,"Yes, received Tdap",No,No
401722,Washington,Female,Very good,0.0,NaN,Within past year (anytime less than 12 months ...,No,8.0,None of them,No,No,No,No,No,No,Yes,No,No,No,No,No,No,No,No,No,Never smoked,Not at all (right now),Yes,Hispanic,Age 35 to 39,1.57,46.72,18.84,Yes,Yes,Yes,NaN,"No, did not receive any tetanus shot in the pa...",No,No
430111,Wisconsin,Male,Very good,0.0,0.0,Within past year (anytime less than 12 months ...,Yes,7.0,None of them,No,No,No,No,No,No,No,No,No,Yes,No,No,No,No,No,No,Never smoked,Never used e-cigarettes in my entire life,Yes,"White only, Non-Hispanic",Age 55 to 59,1.83,90.72,27.12,No,Yes,Yes,Yes,"Yes, received Tdap",No,Tested positive using home test without a heal...
44248,Colorado,Female,Excellent,0.0,0.0,Within past year (anytime less than 12 months ...,Yes,9.0,None of them,No,No,No,Yes,No,No,No,No,No,No,No,No,No,No,No,No,Former smoker,Never used e-cigarettes in my entire life,No,"White only, Non-Hispanic",Age 65 to 69,1.60,73.94,28.87,Yes,No,Yes,Yes,"Yes, received tetanus shot but not sure what type",No,Yes
364699,Utah,Female,Fair,7.0,3.0,Within past year (anytime less than 12 months ...,Yes,7.0,None of them,No,No,Yes,No,No,No,No,No,Yes,No,Yes,Yes,Yes,No,Yes,Yes,Never smoked,Never used e-cigarettes in my entire life,NaN,"White only, Non-Hispanic",Age 45 to 49,1.63,80.74,30.55,No,No,Yes,Yes,"Yes, received Tdap",No,No
60040,District of Columbia,Female,Good,1.0,0.0,Within past year (anytime less than 12 months ...,Yes,6.0,None of them,No,No,No,Yes,No,No,No,No,Yes,No,No,No,No,No,No,No,Former smoker,Never used e-cigarettes in my entire life,No,"White only, Non-Hispanic",Age 65 to 69,1.80,106.59,32.78,Yes,Yes,Yes,No,"Yes, received Tdap",No,No


**Summarize dataset**

In [ ]:
summarize_dataframe(df)

Dataset shape:
Rows: 445132, Columns: 40

Summary:
                           Non-Null Count  Null Count    Dtype  Unique Values
State                              445132           0   object             54
Sex                                445132           0   object              2
GeneralHealth                      443934        1198   object              5
PhysicalHealthDays                 434205       10927  float64             31
MentalHealthDays                   436065        9067  float64             31
LastCheckupTime                    436824        8308   object              4
PhysicalActivities                 444039        1093   object              2
SleepHours                         439679        5453  float64             24
RemovedTeeth                       433772       11360   object              4
HadHeartAttack                     442067        3065   object              2
HadAngina                          440727        4405   object              2
HadStroke    

,Non-Null Count,Null Count,Dtype,Unique Values
State,445132,0,object,54
Sex,445132,0,object,2
GeneralHealth,443934,1198,object,5
PhysicalHealthDays,434205,10927,float64,31
MentalHealthDays,436065,9067,float64,31
LastCheckupTime,436824,8308,object,4
PhysicalActivities,444039,1093,object,2
SleepHours,439679,5453,float64,24
RemovedTeeth,433772,11360,object,4
HadHeartAttack,442067,3065,object,2


**Show unique value**

In [ ]:
show_unique_values(df)


Column: State
Unique values (max 40 shown): ['Alabama' 'Alaska' 'Arizona' 'Arkansas' 'California' 'Colorado'
 'Connecticut' 'Delaware' 'District of Columbia' 'Florida' 'Georgia'
 'Hawaii' 'Idaho' 'Illinois' 'Indiana' 'Iowa' 'Kansas' 'Kentucky'
 'Louisiana' 'Maine' 'Maryland' 'Massachusetts' 'Michigan' 'Minnesota'
 'Mississippi' 'Missouri' 'Montana' 'Nebraska' 'Nevada' 'New Hampshire'
 'New Jersey' 'New Mexico' 'New York' 'North Carolina' 'North Dakota'
 'Ohio' 'Oklahoma' 'Oregon' 'Pennsylvania' 'Rhode Island']
Total unique values: 54

Column: Sex
Unique values (max 40 shown): ['Female' 'Male']
Total unique values: 2

Column: GeneralHealth
Unique values (max 40 shown): ['Very good' 'Excellent' 'Fair' 'Poor' 'Good' nan]
Total unique values: 5

Column: PhysicalHealthDays
Unique values (max 40 shown): [ 0.  2.  1.  8.  5. 30.  4. 23. 14. nan 15.  3. 10.  7. 25.  6. 21. 20.
 29. 16.  9. 27. 28. 12. 13. 11. 26. 17. 24. 19. 18. 22.]
Total unique values: 31

Column: MentalHealthDays
Unique va

# **1. DATA PREPARATION**

## **1.1 Load original dataset**

In [ ]:
# Mount Google Drive
drive.mount('/content/drive')

# Load dataset
if os.path.exists(GDRIVE_ORIGINAL_FILE_PATH):
    print(f"Loading from Google Drive: {GDRIVE_ORIGINAL_FILE_PATH}")
    df = pd.read_csv(GDRIVE_ORIGINAL_FILE_PATH)
else:
    print(f"Original dataset not found in Google Drive: {GDRIVE_ORIGINAL_FILE_PATH}\nRun Download Section")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Loading from Google Drive: /content/drive/MyDrive/dicoding_capstone/datasets/1.original.csv


**Sample**

In [ ]:
df.sample(n=15)

,State,Sex,GeneralHealth,PhysicalHealthDays,MentalHealthDays,LastCheckupTime,PhysicalActivities,SleepHours,RemovedTeeth,HadHeartAttack,HadAngina,HadStroke,HadAsthma,HadSkinCancer,HadCOPD,HadDepressiveDisorder,HadKidneyDisease,HadArthritis,HadDiabetes,DeafOrHardOfHearing,BlindOrVisionDifficulty,DifficultyConcentrating,DifficultyWalking,DifficultyDressingBathing,DifficultyErrands,SmokerStatus,ECigaretteUsage,ChestScan,RaceEthnicityCategory,AgeCategory,HeightInMeters,WeightInKilograms,BMI,AlcoholDrinkers,HIVTesting,FluVaxLast12,PneumoVaxEver,TetanusLast10Tdap,HighRiskLastYear,CovidPos
362663,Utah,Female,Very good,30.0,0.0,Within past year (anytime less than 12 months ...,Yes,7.0,None of them,No,No,No,No,No,No,Yes,No,No,No,No,No,No,No,No,No,Never smoked,Never used e-cigarettes in my entire life,No,"White only, Non-Hispanic",Age 55 to 59,1.68,81.65,29.05,No,Yes,Yes,No,"No, did not receive any tetanus shot in the pa...",No,Yes
137814,Kentucky,Female,Very good,30.0,2.0,Within past 5 years (2 years but less than 5 y...,No,6.0,1 to 5,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Current smoker - now smokes every day,Use them every day,No,"White only, Non-Hispanic",Age 25 to 29,1.63,97.52,36.90,No,No,No,No,"Yes, received Tdap",No,Yes
346751,Texas,Male,Very good,0.0,15.0,Within past 2 years (1 year but less than 2 ye...,Yes,4.0,None of them,No,No,No,No,Yes,No,Yes,No,Yes,No,No,No,No,No,No,No,Former smoker,Never used e-cigarettes in my entire life,Yes,"White only, Non-Hispanic",Age 75 to 79,1.63,87.09,32.96,Yes,No,Yes,Yes,"Yes, received tetanus shot but not sure what type",No,No
322528,Rhode Island,Male,Very good,0.0,0.0,Within past year (anytime less than 12 months ...,Yes,8.0,None of them,No,No,No,No,No,No,Yes,No,No,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"White only, Non-Hispanic",Age 50 to 54,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
293525,Ohio,Female,Excellent,0.0,10.0,Within past year (anytime less than 12 months ...,Yes,6.0,None of them,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Never smoked,Never used e-cigarettes in my entire life,No,"Black only, Non-Hispanic",Age 30 to 34,1.63,57.15,21.63,No,Yes,Yes,No,"Yes, received Tdap",No,Yes
140626,Louisiana,Female,Good,0.0,0.0,Within past year (anytime less than 12 months ...,No,NaN,None of them,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Never smoked,Never used e-cigarettes in my entire life,No,"Black only, Non-Hispanic",Age 35 to 39,1.63,70.76,26.78,Yes,No,Yes,No,NaN,No,Yes
241225,New Hampshire,Female,Very good,0.0,0.0,Within past 5 years (2 years but less than 5 y...,Yes,6.0,1 to 5,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Never smoked,Never used e-cigarettes in my entire life,No,"White only, Non-Hispanic",Age 60 to 64,1.68,104.33,37.12,No,No,No,No,"Yes, received tetanus shot, but not Tdap",No,No
55627,Connecticut,Male,Excellent,0.0,0.0,Within past year (anytime less than 12 months ...,Yes,7.0,None of them,No,No,No,No,Yes,No,No,No,Yes,No,No,No,No,No,No,No,Never smoked,Never used e-cigarettes in my entire life,Yes,"White only, Non-Hispanic",Age 70 to 74,1.80,83.91,25.80,Yes,Yes,Yes,Yes,"Yes, received tetanus shot, but not Tdap",No,Tested positive using home test without a heal...
408957,Washington,Female,Very good,0.0,10.0,Within past year (anytime less than 12 months ...,Yes,7.0,None of them,No,No,No,Yes,No,No,No,No,No,No,No,No,No,No,No,No,Never smoked,Never used e-cigarettes in my entire life,NaN,"White only, Non-Hispanic",Age 25 to 29,1.68,90.72,32.28,Yes,No,No,NaN,"Yes, received Tdap",No,No
216752,Missouri,Male,Excellent,0.0,0.0,5 or more years ago,No,7.0,1 to 5,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Current smoker - now smokes every day,Never used e-cigarettes in my entire life,No,"White only, Non-Hispanic",Age 45 to 49,1.83,74.84,22.38,Yes,No,No,No,"Yes, received tetanus shot, but not Tdap",No,No


**Summary**

In [ ]:
summarize_dataframe(df)

Dataset shape:
Rows: 445132, Columns: 40

Summary:
                           Non-Null Count  Null Count    Dtype  Unique Values
State                              445132           0   object             54
Sex                                445132           0   object              2
GeneralHealth                      443934        1198   object              5
PhysicalHealthDays                 434205       10927  float64             31
MentalHealthDays                   436065        9067  float64             31
LastCheckupTime                    436824        8308   object              4
PhysicalActivities                 444039        1093   object              2
SleepHours                         439679        5453  float64             24
RemovedTeeth                       433772       11360   object              4
HadHeartAttack                     442067        3065   object              2
HadAngina                          440727        4405   object              2
HadStroke    

,Non-Null Count,Null Count,Dtype,Unique Values
State,445132,0,object,54
Sex,445132,0,object,2
GeneralHealth,443934,1198,object,5
PhysicalHealthDays,434205,10927,float64,31
MentalHealthDays,436065,9067,float64,31
LastCheckupTime,436824,8308,object,4
PhysicalActivities,444039,1093,object,2
SleepHours,439679,5453,float64,24
RemovedTeeth,433772,11360,object,4
HadHeartAttack,442067,3065,object,2


**Unique values**

In [ ]:
show_unique_values(df)


Column: State
Unique values (max 40 shown): ['Alabama' 'Alaska' 'Arizona' 'Arkansas' 'California' 'Colorado'
 'Connecticut' 'Delaware' 'District of Columbia' 'Florida' 'Georgia'
 'Hawaii' 'Idaho' 'Illinois' 'Indiana' 'Iowa' 'Kansas' 'Kentucky'
 'Louisiana' 'Maine' 'Maryland' 'Massachusetts' 'Michigan' 'Minnesota'
 'Mississippi' 'Missouri' 'Montana' 'Nebraska' 'Nevada' 'New Hampshire'
 'New Jersey' 'New Mexico' 'New York' 'North Carolina' 'North Dakota'
 'Ohio' 'Oklahoma' 'Oregon' 'Pennsylvania' 'Rhode Island']
Total unique values: 54

Column: Sex
Unique values (max 40 shown): ['Female' 'Male']
Total unique values: 2

Column: GeneralHealth
Unique values (max 40 shown): ['Very good' 'Excellent' 'Fair' 'Poor' 'Good' nan]
Total unique values: 5

Column: PhysicalHealthDays
Unique values (max 40 shown): [ 0.  2.  1.  8.  5. 30.  4. 23. 14. nan 15.  3. 10.  7. 25.  6. 21. 20.
 29. 16.  9. 27. 28. 12. 13. 11. 26. 17. 24. 19. 18. 22.]
Total unique values: 31

Column: MentalHealthDays
Unique va

## **1.2 Handle duplicate**

**Check for duplicate**

In [ ]:
print(f"Duplicate rows:\n{df.duplicated().sum()}")

Duplicate rows:
157


**Drop Duplicates**

In [ ]:
df_no_duplicate = df.copy()

df_no_duplicate = df_no_duplicate.drop_duplicates()
print(f"\nDuplicate columns after processing:\n{df_no_duplicate.duplicated().sum()}")

print(f"\nSaving to Google Drive: {GDRIVE_NO_DUPLICATE_DATASET_PATH}")
df_no_duplicate.to_csv(GDRIVE_NO_DUPLICATE_DATASET_PATH, index=False)


Duplicate columns after processing:
0

Saving to Google Drive: /content/drive/MyDrive/dicoding_capstone/datasets/2.no_duplicate.csv


## **1.3 Handle irrelevant feature**

**Immediate drop irrelevant columns**

Reason:
1. `State`: too many unique values and unreliable. Typically used only for mapping; any genetic-related factors (are already/can be) represented by `RaceEthnicityCategory`.
2. `ChestScan`: irrelevant. Based on the documentation, this question only asks whether the respondent has ever had a CT scan, serving merely as a gateway to more specific CT scan results.
3. `HighRiskLastYear`: redundant. This column is an input used to create `HIVTesting` Column.
4. `LastCheckupTime`: not so important

In [ ]:
df_no_irrelevant = df_no_duplicate.copy()

df_no_irrelevant = df_no_irrelevant.drop(columns=["State", "ChestScan", "HighRiskLastYear", "LastCheckupTime"])

summarize_dataframe(df_no_irrelevant)

print(f"\nSaving to Google Drive: {GDRIVE_NO_IRRELEVANT_DATASET_PATH}")
df_no_irrelevant.to_csv(GDRIVE_NO_IRRELEVANT_DATASET_PATH, index=False)

Dataset shape:
Rows: 444975, Columns: 36

Summary:
                           Non-Null Count  Null Count    Dtype  Unique Values
Sex                                444975           0   object              2
GeneralHealth                      443782        1193   object              5
PhysicalHealthDays                 434053       10922  float64             31
MentalHealthDays                   435913        9062  float64             31
PhysicalActivities                 443887        1088   object              2
SleepHours                         439527        5448  float64             24
RemovedTeeth                       433620       11355   object              4
HadHeartAttack                     441915        3060   object              2
HadAngina                          440575        4400   object              2
HadStroke                          443423        1552   object              2
HadAsthma                          443207        1768   object              2
HadSkinCancer

## **1.4 Feature evaluation**

### **1.4.1 Binary**

**List of binary columns**

In [ ]:
binary_cols = [
    'Sex',
    'PhysicalActivities',
    'HadAsthma',
    'HadSkinCancer',
    'HadCOPD',
    'HadDepressiveDisorder',
    'HadKidneyDisease',
    'HadArthritis',
    'DeafOrHardOfHearing',
    'BlindOrVisionDifficulty',
    'DifficultyConcentrating',
    'DifficultyWalking',
    'DifficultyDressingBathing',
    'DifficultyErrands',
    'ChestScan',
    'AlcoholDrinkers',
    'HIVTesting',
    'FluVaxLast12',
    'PneumoVaxEver',
]

**`Sex` is the only feature with different kind of value in the binary columns**

In [ ]:
df_fixed_binary = df_no_irrelevant.copy()

df_fixed_binary = df_fixed_binary.rename(columns={'Sex': 'IsMale'})
df_fixed_binary['IsMale'] = df_fixed_binary['IsMale'].map({'Male': 'Yes', 'Female': 'No'})

index_to_replace = binary_cols.index('Sex')
binary_cols[index_to_replace] = 'IsMale'

show_unique_values(df_fixed_binary[['IsMale']])

print(f"\nSaving to Google Drive: {GDRIVE_FIXED_BINARY_DATASET_PATH}")

df_fixed_binary.to_csv(GDRIVE_FIXED_BINARY_DATASET_PATH, index=False)


Column: IsMale
Unique values (max 40 shown): ['No' 'Yes']
Total unique values: 2

Saving to Google Drive: /content/drive/MyDrive/dicoding_capstone/datasets/4.fixed_binary.csv


### **1.4.2 Ordinal**

**List of ordinal columns**

In [ ]:
ordinal_cols = [
    'GeneralHealth',
    'RemovedTeeth',
    'SmokerStatus',
    'ECigaretteUsage',
    'AgeCategory'
]

**Values to change**

In [ ]:
removed_teeth_map = {
    'None of them': 'none',
    '1 to 5': 'less then 6',
    '6 or more, but not all': 'more than 6, but not all',
    'All': 'all'
}

smoker_status_map = {
    'Never smoked': 'never',
    'Former smoker': 'used to',
    'Current smoker - now smokes some days': 'occasionally',
    'Current smoker - now smokes every day': 'daily'
}


ecigarette_usage_map = {
    'Never used e-cigarettes in my entire life': 'never',
    'Not at all (right now)': 'not currently',
    'Use them some days': 'occasionally',
    'Use them every day': 'daily'
}

age_category_map = {
    'Age 18 to 24': '18-24 years',
    'Age 25 to 29': '25-29 years',
    'Age 30 to 34': '30-34 years',
    'Age 35 to 39': '35-39 years',
    'Age 40 to 44': '40-44 years',
    'Age 45 to 49': '45-49 years',
    'Age 50 to 54': '50-54 years',
    'Age 55 to 59': '55-59 years',
    'Age 60 to 64': '60-64 years',
    'Age 65 to 69': '65-69 years',
    'Age 70 to 74': '70-74 years',
    'Age 75 to 79': '75-79 years',
    'Age 80 or older': '80+ years'
}

ordinal_maps = {
    'RemovedTeeth': removed_teeth_map,
    'SmokerStatus': smoker_status_map,
    'ECigaretteUsage': ecigarette_usage_map,
    'AgeCategory': age_category_map
}

df_fixed_ordinal = df_fixed_binary.copy()

for col, mapping in ordinal_maps.items():
    df_fixed_ordinal[col] = df_fixed_ordinal[col].map(mapping)

show_unique_values(df_fixed_ordinal[ordinal_cols])

df_fixed_ordinal.to_csv(GDRIVE_FIXED_ORDINAL_DATASET_PATH, index=False)


Column: GeneralHealth
Unique values (max 40 shown): ['Very good' 'Excellent' 'Fair' 'Poor' 'Good' nan]
Total unique values: 5

Column: RemovedTeeth
Unique values (max 40 shown): [nan 'none' 'less then 6' 'more than 6, but not all' 'all']
Total unique values: 4

Column: SmokerStatus
Unique values (max 40 shown): ['never' 'occasionally' 'used to' nan 'daily']
Total unique values: 4

Column: ECigaretteUsage
Unique values (max 40 shown): ['not currently' 'never' nan 'daily' 'occasionally']
Total unique values: 4

Column: AgeCategory
Unique values (max 40 shown): ['80+ years' '55-59 years' nan '40-44 years' '75-79 years' '70-74 years'
 '65-69 years' '60-64 years' '50-54 years' '45-49 years' '35-39 years'
 '25-29 years' '30-34 years' '18-24 years']
Total unique values: 13


### **1.4.3 Nominal**

**List of nominal columns**

In [ ]:
nominal_cols = [
    'HadDiabetes',            # 4
    'RaceEthnicityCategory',  # 5
    'TetanusLast10Tdap',      # 4
    'CovidPos'                # 3
]

**Values to change**

In [ ]:
diabetes_map = {
    'No': 'no',
    'Yes': 'yes',
    'No, pre-diabetes or borderline diabetes': 'borderline',
    'Yes, but only during pregnancy (female)': 'pregnancy only'
}

race_map = {
    'White only, Non-Hispanic': 'white',
    'Black only, Non-Hispanic': 'black',
    'Hispanic': 'hispanic',
    'Other race only, Non-Hispanic': 'other',
    'Multiracial, Non-Hispanic': 'multiracial'
}

tetanus_map = {
    'No, did not receive any tetanus shot in the past 10 years': 'no',
    'Yes, received Tdap': 'yes_tdap',
    'Yes, received tetanus shot, but not Tdap': 'yes_tetanus_only',
    'Yes, received tetanus shot but not sure what type': 'yes_unsure'
}

covid_map = {
    'No': 'no',
    'Yes': 'yes',
    'Tested positive using home test without a health professional': 'yes_home_test'
}

nominal_maps = {
    'HadDiabetes': diabetes_map,
    'RaceEthnicityCategory': race_map,
    'TetanusLast10Tdap': tetanus_map,
    'CovidPos': covid_map
}

df_fixed_nominal = df_fixed_ordinal.copy()

for col, mapping in nominal_maps.items():
    df_fixed_nominal[col] = df_fixed_nominal[col].map(mapping)

show_unique_values(df_fixed_nominal[nominal_cols])

df_fixed_nominal.to_csv(GDRIVE_FIXED_NOMINAL_DATASET_PATH, index=False)


Column: HadDiabetes
Unique values (max 40 shown): ['yes' 'no' 'borderline' nan 'pregnancy only']
Total unique values: 4

Column: RaceEthnicityCategory
Unique values (max 40 shown): ['white' 'black' 'other' 'multiracial' nan 'hispanic']
Total unique values: 5

Column: TetanusLast10Tdap
Unique values (max 40 shown): ['yes_unsure' 'no' nan 'yes_tdap' 'yes_tetanus_only']
Total unique values: 4

Column: CovidPos
Unique values (max 40 shown): ['no' 'yes' nan 'yes_home_test']
Total unique values: 3


In [ ]:
df_fixed_nominal.sample(n=15)

,IsMale,GeneralHealth,PhysicalHealthDays,MentalHealthDays,PhysicalActivities,SleepHours,RemovedTeeth,HadHeartAttack,HadAngina,HadStroke,HadAsthma,HadSkinCancer,HadCOPD,HadDepressiveDisorder,HadKidneyDisease,HadArthritis,HadDiabetes,DeafOrHardOfHearing,BlindOrVisionDifficulty,DifficultyConcentrating,DifficultyWalking,DifficultyDressingBathing,DifficultyErrands,SmokerStatus,ECigaretteUsage,RaceEthnicityCategory,AgeCategory,HeightInMeters,WeightInKilograms,BMI,AlcoholDrinkers,HIVTesting,FluVaxLast12,PneumoVaxEver,TetanusLast10Tdap,CovidPos
32205,Yes,Good,0.0,0.0,Yes,6.0,none,No,No,No,No,No,No,No,No,No,no,No,No,No,No,No,No,used to,not currently,white,35-39 years,1.78,88.45,27.98,Yes,No,No,No,no,yes
160099,No,Good,3.0,5.0,Yes,8.0,less then 6,No,No,No,No,No,No,No,No,No,no,No,No,No,Yes,No,Yes,daily,not currently,hispanic,50-54 years,1.52,61.23,26.37,NaN,NaN,NaN,NaN,NaN,NaN
223571,Yes,Excellent,0.0,0.0,Yes,8.0,none,No,No,No,No,No,No,No,No,No,no,No,No,No,No,No,No,never,never,white,50-54 years,1.91,90.72,25.00,Yes,Yes,No,No,yes_unsure,no
252031,Yes,Very good,20.0,25.0,Yes,1.0,none,No,No,No,No,No,No,No,No,No,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,other,35-39 years,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
319629,No,Very good,3.0,0.0,Yes,7.0,less then 6,No,No,No,Yes,Yes,No,No,Yes,No,no,No,No,No,No,No,No,used to,never,white,60-64 years,1.70,83.01,28.66,Yes,Yes,Yes,Yes,yes_tdap,yes
196671,Yes,Very good,3.0,0.0,Yes,7.0,none,No,No,No,No,No,No,No,No,No,no,No,No,No,No,No,No,never,never,white,65-69 years,1.78,88.00,27.84,Yes,No,Yes,Yes,yes_unsure,yes
101750,Yes,Good,0.0,2.0,Yes,NaN,"more than 6, but not all",No,No,No,No,No,No,Yes,No,Yes,no,Yes,No,Yes,Yes,Yes,No,used to,never,white,60-64 years,1.80,90.72,27.89,Yes,NaN,No,No,yes_unsure,no
165904,No,Good,14.0,20.0,Yes,7.0,less then 6,No,No,No,Yes,No,No,Yes,No,No,pregnancy only,No,No,Yes,No,No,No,used to,occasionally,multiracial,30-34 years,1.70,61.23,21.14,Yes,Yes,Yes,No,yes_tdap,yes
58034,No,Very good,0.0,0.0,Yes,8.0,none,No,No,No,No,No,No,No,No,Yes,no,No,No,No,No,No,No,never,never,NaN,NaN,NaN,NaN,NaN,NaN,No,Yes,No,yes_tdap,no
246409,Yes,Good,0.0,0.0,Yes,8.0,none,No,No,No,No,No,No,No,No,No,no,No,No,No,No,No,No,never,never,hispanic,45-49 years,1.69,81.65,28.59,Yes,Yes,No,No,no,no


### **1.4.4 Create target**

In [ ]:
source_col = ['HadHeartAttack', 'HadAngina', 'HadStroke']

def categorize(row):
    conditions = []
    if str(row['HadHeartAttack']).lower() == "yes":
        conditions.append("HeartAttack")
    if str(row['HadAngina']).lower() == "yes":
        conditions.append("Angina")
    if str(row['HadStroke']).lower() == "yes":
        conditions.append("Stroke")
    return " & ".join(conditions) if conditions else "None"

df_with_target = df_fixed_nominal.copy()

df_with_target["Condition"] = df_with_target[source_col].apply(categorize, axis=1)

combo_summary = df_with_target["Condition"].value_counts().to_frame(name="Count")
combo_summary["Percentage"] = (df_with_target["Condition"].value_counts(normalize=True) * 100).round(2)

print(combo_summary)

# create target
df_with_target["HeartDisease"] = df_with_target[source_col].apply(
    lambda row: 1 if "Yes" in row.values else 0, axis=1
)

df_with_target = df_with_target.drop(columns=source_col + ["Condition"])

print("\nUnique value count:")
print(df_with_target["HeartDisease"].value_counts())
print("\nUnique value percentage:")
print((df_with_target["HeartDisease"].value_counts(normalize=True) * 100).round(2))
print(f"\nUnique value:\n{df_with_target['HeartDisease'].unique()}")

print(f"\nSaving to Google Drive: {GDRIVE_WITH_TARGET_FILE_PATH}")
df_with_target.to_csv(GDRIVE_WITH_TARGET_FILE_PATH, index=False)

                                Count  Percentage
Condition                                        
None                           392720       88.26
Angina                          12845        2.89
Stroke                          12504        2.81
HeartAttack                     10832        2.43
HeartAttack & Angina             9340        2.10
HeartAttack & Angina & Stroke    2568        0.58
HeartAttack & Stroke             2368        0.53
Angina & Stroke                  1798        0.40

Unique value count:
HeartDisease
0    392720
1     52255
Name: count, dtype: int64

Unique value percentage:
HeartDisease
0    88.26
1    11.74
Name: proportion, dtype: float64

Unique value:
[0 1]

Saving to Google Drive: /content/drive/MyDrive/dicoding_capstone/datasets/7.with_target.csv


### **1.4.5 Apply lowercase on object data type**

In [ ]:
df_lowercase_object = df_with_target.copy()

for col in df_lowercase_object.columns:
    if df_lowercase_object[col].dtype == 'object':
        df_lowercase_object[col] = df_lowercase_object[col].str.lower()

show_unique_values(df_lowercase_object.select_dtypes(include=['object']))

print(f"\nSaving to Google Drive: {GDRIVE_LOWERCASE_OBJECT_DATASET_PATH}")
df_lowercase_object.to_csv(GDRIVE_LOWERCASE_OBJECT_DATASET_PATH, index=False)


Column: IsMale
Unique values (max 40 shown): ['no' 'yes']
Total unique values: 2

Column: GeneralHealth
Unique values (max 40 shown): ['very good' 'excellent' 'fair' 'poor' 'good' nan]
Total unique values: 5

Column: PhysicalActivities
Unique values (max 40 shown): ['no' 'yes' nan]
Total unique values: 2

Column: RemovedTeeth
Unique values (max 40 shown): [nan 'none' 'less then 6' 'more than 6, but not all' 'all']
Total unique values: 4

Column: HadAsthma
Unique values (max 40 shown): ['no' 'yes' nan]
Total unique values: 2

Column: HadSkinCancer
Unique values (max 40 shown): ['no' 'yes' nan]
Total unique values: 2

Column: HadCOPD
Unique values (max 40 shown): ['no' 'yes' nan]
Total unique values: 2

Column: HadDepressiveDisorder
Unique values (max 40 shown): ['no' 'yes' nan]
Total unique values: 2

Column: HadKidneyDisease
Unique values (max 40 shown): ['no' 'yes' nan]
Total unique values: 2

Column: HadArthritis
Unique values (max 40 shown): ['no' 'yes' nan]
Total unique values: 2


## **1.5 Review**

In [ ]:
df_lowercase_object.sample(n=15)

,IsMale,GeneralHealth,PhysicalHealthDays,MentalHealthDays,PhysicalActivities,SleepHours,RemovedTeeth,HadAsthma,HadSkinCancer,HadCOPD,HadDepressiveDisorder,HadKidneyDisease,HadArthritis,HadDiabetes,DeafOrHardOfHearing,BlindOrVisionDifficulty,DifficultyConcentrating,DifficultyWalking,DifficultyDressingBathing,DifficultyErrands,SmokerStatus,ECigaretteUsage,RaceEthnicityCategory,AgeCategory,HeightInMeters,WeightInKilograms,BMI,AlcoholDrinkers,HIVTesting,FluVaxLast12,PneumoVaxEver,TetanusLast10Tdap,CovidPos,HeartDisease
194455,yes,very good,2.0,3.0,yes,7.0,none,no,no,no,no,no,no,no,no,no,no,no,no,no,used to,never,white,40-44 years,1.83,88.45,26.45,yes,no,yes,no,yes_unsure,yes_home_test,0
32952,yes,good,0.0,0.0,yes,8.0,none,no,no,no,no,no,no,no,no,no,no,yes,no,no,never,never,hispanic,55-59 years,1.85,99.79,29.16,yes,no,yes,no,yes_unsure,no,1
17560,no,very good,0.0,0.0,yes,8.0,less then 6,no,no,no,no,no,yes,no,no,no,no,no,no,no,used to,never,white,80+ years,1.55,47.63,19.84,yes,no,no,no,no,no,0
164855,yes,very good,0.0,0.0,no,6.0,less then 6,no,no,no,no,no,no,no,no,no,NaN,NaN,NaN,NaN,NaN,NaN,black,45-49 years,1.83,146.06,43.67,NaN,NaN,NaN,NaN,NaN,NaN,1
326592,no,fair,3.0,0.0,yes,8.0,none,no,no,no,no,no,no,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,black,18-24 years,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
238371,yes,fair,15.0,NaN,no,NaN,NaN,no,no,no,no,no,no,no,no,no,yes,no,no,yes,daily,not currently,white,60-64 years,1.83,83.91,25.09,yes,no,no,no,no,NaN,0
359357,no,fair,0.0,0.0,no,5.0,"more than 6, but not all",no,yes,no,no,no,yes,yes,yes,no,yes,yes,no,no,never,never,white,80+ years,1.63,81.65,30.90,no,no,yes,yes,yes_unsure,no,0
350912,yes,very good,0.0,0.0,yes,8.0,less then 6,no,no,no,no,no,no,no,no,no,no,no,no,no,never,never,hispanic,60-64 years,1.70,83.91,28.97,yes,no,yes,no,yes_tdap,no,0
380829,no,fair,15.0,5.0,no,7.0,less then 6,no,no,no,no,yes,yes,yes,no,no,no,yes,no,no,never,never,white,70-74 years,1.63,95.25,36.05,no,NaN,yes,no,yes_tdap,no,0
341005,yes,fair,30.0,0.0,yes,6.0,none,yes,no,yes,yes,no,yes,no,yes,yes,yes,yes,no,no,occasionally,never,multiracial,65-69 years,1.83,86.18,25.77,yes,yes,yes,yes,yes_unsure,no,1


In [ ]:
summarize_dataframe(df_lowercase_object)

Dataset shape:
Rows: 444975, Columns: 34

Summary:
                           Non-Null Count  Null Count    Dtype  Unique Values
IsMale                             444975           0   object              2
GeneralHealth                      443782        1193   object              5
PhysicalHealthDays                 434053       10922  float64             31
MentalHealthDays                   435913        9062  float64             31
PhysicalActivities                 443887        1088   object              2
SleepHours                         439527        5448  float64             24
RemovedTeeth                       433620       11355   object              4
HadAsthma                          443207        1768   object              2
HadSkinCancer                      441837        3138   object              2
HadCOPD                            442761        2214   object              2
HadDepressiveDisorder              442168        2807   object              2
HadKidneyDise

,Non-Null Count,Null Count,Dtype,Unique Values
IsMale,444975,0,object,2
GeneralHealth,443782,1193,object,5
PhysicalHealthDays,434053,10922,float64,31
MentalHealthDays,435913,9062,float64,31
PhysicalActivities,443887,1088,object,2
SleepHours,439527,5448,float64,24
RemovedTeeth,433620,11355,object,4
HadAsthma,443207,1768,object,2
HadSkinCancer,441837,3138,object,2
HadCOPD,442761,2214,object,2


In [ ]:
show_unique_values(df_lowercase_object)


Column: IsMale
Unique values (max 40 shown): ['no' 'yes']
Total unique values: 2

Column: GeneralHealth
Unique values (max 40 shown): ['very good' 'excellent' 'fair' 'poor' 'good' nan]
Total unique values: 5

Column: PhysicalHealthDays
Unique values (max 40 shown): [ 0.  2.  1.  8.  5. 30.  4. 23. 14. nan 15.  3. 10.  7. 25.  6. 21. 20.
 29. 16.  9. 27. 28. 12. 13. 11. 26. 17. 24. 19. 18. 22.]
Total unique values: 31

Column: MentalHealthDays
Unique values (max 40 shown): [ 0.  3.  9.  5. 15. 20. 14. 10. 18.  1. nan  2. 30.  4.  6.  7. 25.  8.
 22. 29. 27. 21. 12. 28. 16. 13. 26. 17. 11. 23. 19. 24.]
Total unique values: 31

Column: PhysicalActivities
Unique values (max 40 shown): ['no' 'yes' nan]
Total unique values: 2

Column: SleepHours
Unique values (max 40 shown): [ 8.  6.  5.  7.  9.  4. 10.  1. 12. nan 18.  3.  2. 11. 16. 15. 13. 14.
 20. 23. 17. 24. 22. 19. 21.]
Total unique values: 24

Column: RemovedTeeth
Unique values (max 40 shown): [nan 'none' 'less then 6' 'more than 6, 

# **2. PIPELINE**

## **2.1 Load prepared dataset**

In [ ]:
# Mount Google Drive
drive.mount('/content/drive')

# Load dataset
if os.path.exists(GDRIVE_LOWERCASE_OBJECT_DATASET_PATH):
    print(f"Loading from Google Drive: {GDRIVE_LOWERCASE_OBJECT_DATASET_PATH}")
    df_prepared = pd.read_csv(GDRIVE_LOWERCASE_OBJECT_DATASET_PATH)
else:
    print(f"Prepared dataset not found in Google Drive: {GDRIVE_LOWERCASE_OBJECT_DATASET_PATH}\nRun Data Preparation")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Loading from Google Drive: /content/drive/MyDrive/dicoding_capstone/datasets/8.lowercase_object.csv


**Sample**

In [ ]:
df_prepared.sample(n=5)

,IsMale,GeneralHealth,PhysicalHealthDays,MentalHealthDays,PhysicalActivities,SleepHours,RemovedTeeth,HadAsthma,HadSkinCancer,HadCOPD,HadDepressiveDisorder,HadKidneyDisease,HadArthritis,HadDiabetes,DeafOrHardOfHearing,BlindOrVisionDifficulty,DifficultyConcentrating,DifficultyWalking,DifficultyDressingBathing,DifficultyErrands,SmokerStatus,ECigaretteUsage,RaceEthnicityCategory,AgeCategory,HeightInMeters,WeightInKilograms,BMI,AlcoholDrinkers,HIVTesting,FluVaxLast12,PneumoVaxEver,TetanusLast10Tdap,CovidPos,HeartDisease
392476,yes,good,0.0,0.0,yes,7.0,none,no,no,no,yes,no,no,no,no,no,no,no,no,no,never,never,white,60-64 years,1.78,84.82,26.83,yes,no,yes,no,NaN,no,0
322430,no,good,0.0,0.0,yes,6.0,less then 6,yes,no,no,no,no,no,yes,no,no,no,no,no,no,used to,never,white,65-69 years,1.57,113.40,45.73,no,no,yes,yes,no,no,0
389468,no,very good,0.0,0.0,yes,6.0,none,no,no,no,no,no,no,no,no,no,no,no,no,no,never,never,white,80+ years,1.60,54.43,21.26,no,no,yes,no,no,no,0
124295,no,fair,30.0,30.0,yes,3.0,"more than 6, but not all",no,no,no,no,no,no,yes,no,no,yes,no,no,no,daily,not currently,other,60-64 years,1.75,81.65,26.58,no,no,no,yes,no,no,0
86009,no,good,0.0,0.0,yes,6.0,less then 6,no,no,no,no,no,yes,yes,no,no,no,no,no,no,used to,never,white,70-74 years,1.73,61.23,20.53,no,no,no,no,no,no,0


**Summary**

In [ ]:
summarize_dataframe(df_prepared)

Dataset shape:
Rows: 444975, Columns: 34

Summary:
                           Non-Null Count  Null Count    Dtype  Unique Values
IsMale                             444975           0   object              2
GeneralHealth                      443782        1193   object              5
PhysicalHealthDays                 434053       10922  float64             31
MentalHealthDays                   435913        9062  float64             31
PhysicalActivities                 443887        1088   object              2
SleepHours                         439527        5448  float64             24
RemovedTeeth                       433620       11355   object              4
HadAsthma                          443207        1768   object              2
HadSkinCancer                      441837        3138   object              2
HadCOPD                            442761        2214   object              2
HadDepressiveDisorder              442168        2807   object              2
HadKidneyDise

,Non-Null Count,Null Count,Dtype,Unique Values
IsMale,444975,0,object,2
GeneralHealth,443782,1193,object,5
PhysicalHealthDays,434053,10922,float64,31
MentalHealthDays,435913,9062,float64,31
PhysicalActivities,443887,1088,object,2
SleepHours,439527,5448,float64,24
RemovedTeeth,433620,11355,object,4
HadAsthma,443207,1768,object,2
HadSkinCancer,441837,3138,object,2
HadCOPD,442761,2214,object,2


**Unique values**

In [ ]:
show_unique_values(df_prepared)


Column: IsMale
Unique values (max 40 shown): ['no' 'yes']
Total unique values: 2

Column: GeneralHealth
Unique values (max 40 shown): ['very good' 'excellent' 'fair' 'poor' 'good' nan]
Total unique values: 5

Column: PhysicalHealthDays
Unique values (max 40 shown): [ 0.  2.  1.  8.  5. 30.  4. 23. 14. nan 15.  3. 10.  7. 25.  6. 21. 20.
 29. 16.  9. 27. 28. 12. 13. 11. 26. 17. 24. 19. 18. 22.]
Total unique values: 31

Column: MentalHealthDays
Unique values (max 40 shown): [ 0.  3.  9.  5. 15. 20. 14. 10. 18.  1. nan  2. 30.  4.  6.  7. 25.  8.
 22. 29. 27. 21. 12. 28. 16. 13. 26. 17. 11. 23. 19. 24.]
Total unique values: 31

Column: PhysicalActivities
Unique values (max 40 shown): ['no' 'yes' nan]
Total unique values: 2

Column: SleepHours
Unique values (max 40 shown): [ 8.  6.  5.  7.  9.  4. 10.  1. 12. nan 18.  3.  2. 11. 16. 15. 13. 14.
 20. 23. 17. 24. 22. 19. 21.]
Total unique values: 24

Column: RemovedTeeth
Unique values (max 40 shown): [nan 'none' 'less then 6' 'more than 6, 

## **2.2 Preprocessing**

**Transformers**

In [ ]:
%%writefile feature_mapper.py
from sklearn.base import BaseEstimator, TransformerMixin

class FeatureMapper(BaseEstimator, TransformerMixin):
    def __init__(self, mappings):
        self.mappings = mappings
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        X_copy = X.copy()
        for col, mapping in self.mappings.items():
            if col in X_copy.columns:
                X_copy[col] = X_copy[col].astype(str).str.lower().map(mapping)
        return X_copy
    def set_output(self, *, transform=None):
        return self

Writing feature_mapper.py


In [ ]:
from feature_mapper import FeatureMapper

**Map**

In [ ]:
# Columns
numeric_cols = ['PhysicalHealthDays', 'MentalHealthDays', 'SleepHours', 'HeightInMeters', 'WeightInKilograms', 'BMI']
ordinal_cols = ['GeneralHealth', 'RemovedTeeth', 'SmokerStatus', 'ECigaretteUsage', 'AgeCategory']
binary_cols = ['IsMale', 'PhysicalActivities', 'HadAsthma', 'HadSkinCancer', 'HadCOPD', 'HadDepressiveDisorder', 'HadKidneyDisease', 'HadArthritis', 'DeafOrHardOfHearing', 'BlindOrVisionDifficulty', 'DifficultyConcentrating', 'DifficultyWalking', 'DifficultyDressingBathing', 'DifficultyErrands', 'AlcoholDrinkers', 'HIVTesting', 'FluVaxLast12', 'PneumoVaxEver']
nominal_cols = ['HadDiabetes', 'RaceEthnicityCategory', 'TetanusLast10Tdap', 'CovidPos']
target_col = ['HeartDisease']

# Ordinal map
general_health_map = {'poor': 0, 'fair': 1, 'good': 2, 'very good': 3, 'excellent': 4}
removed_teeth_map = {'none': 0, 'less then 6': 1, 'more than 6, but not all': 2, 'all': 3}
smoker_status_map = {'never': 0, 'used to': 1, 'occasionally': 2, 'daily': 3}
ecigarette_usage_map = {'never': 0, 'not currently': 1, 'occasionally': 2, 'daily': 3}
age_category_map = {
    '18-24 years': 0, '25-29 years': 1, '30-34 years': 2, '35-39 years': 3,
    '40-44 years': 4, '45-49 years': 5, '50-54 years': 6, '55-59 years': 7,
    '60-64 years': 8, '65-69 years': 9, '70-74 years': 10, '75-79 years': 11,
    '80+ years': 12
}

ordinal_maps = {
    'GeneralHealth': general_health_map,
    'RemovedTeeth': removed_teeth_map,
    'SmokerStatus': smoker_status_map,
    'ECigaretteUsage': ecigarette_usage_map,
    'AgeCategory': age_category_map
}

# Binary map
binary_map = {'yes': 1, 'no': 0}
all_binary_maps = {col: binary_map for col in binary_cols}

**Preprocessor**

In [ ]:
distance_based_preprocessor = ColumnTransformer(transformers=[
    ('num', Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='median')),
        ('scaler', StandardScaler())
    ]), numeric_cols),
    ('ord', Pipeline(steps=[
        ('mapper', FeatureMapper(mappings=ordinal_maps)),
        ('imputer', SimpleImputer(strategy='median')),
        ('scaler', StandardScaler())
    ]), ordinal_cols),
    ('bin', Pipeline(steps=[
        ('mapper', FeatureMapper(mappings=all_binary_maps)),
        ('imputer', SimpleImputer(strategy='most_frequent'))
    ]), binary_cols),
    ('nom', Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='constant', fill_value='unknown')),
        ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False, dtype=int))
    ]), nominal_cols)
], remainder='passthrough', verbose_feature_names_out=False)

tree_based_preprocessor = ColumnTransformer(transformers=[
    ('num', Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='median')),
        ('scaler', StandardScaler())
    ]), numeric_cols),
    ('ord', Pipeline(steps=[
        ('mapper', FeatureMapper(mappings=ordinal_maps)),
        ('imputer', SimpleImputer(strategy='median'))
    ]), ordinal_cols),
    ('bin', Pipeline(steps=[
        ('mapper', FeatureMapper(mappings=all_binary_maps)),
        ('imputer', SimpleImputer(strategy='most_frequent'))
    ]), binary_cols),
    ('nom', Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='constant', fill_value='unknown')),
        ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False, dtype=int))
    ]), nominal_cols)
], remainder='passthrough', verbose_feature_names_out=False)

In [ ]:
# test_pipeline = ImbPipeline(steps=[
#         ('preprocessor', tree_based_preprocessor)
# ])

# test_pipeline.fit_transform(df_prepared)

## **2.3 Create model**

**Check dataset**

In [ ]:
df_prepared.sample(n=1)

,IsMale,GeneralHealth,PhysicalHealthDays,MentalHealthDays,PhysicalActivities,SleepHours,RemovedTeeth,HadAsthma,HadSkinCancer,HadCOPD,HadDepressiveDisorder,HadKidneyDisease,HadArthritis,HadDiabetes,DeafOrHardOfHearing,BlindOrVisionDifficulty,DifficultyConcentrating,DifficultyWalking,DifficultyDressingBathing,DifficultyErrands,SmokerStatus,ECigaretteUsage,RaceEthnicityCategory,AgeCategory,HeightInMeters,WeightInKilograms,BMI,AlcoholDrinkers,HIVTesting,FluVaxLast12,PneumoVaxEver,TetanusLast10Tdap,CovidPos,HeartDisease
313688,no,very good,0.0,10.0,yes,8.0,none,no,no,no,no,no,no,no,no,no,no,no,no,no,never,never,white,18-24 years,1.6,55.34,21.61,yes,no,no,yes,yes_unsure,no,0


**Import Library**

**Split**

In [ ]:
print(df_prepared[['HeartDisease']].value_counts(normalize=False))
print(df_prepared[['HeartDisease']].value_counts(normalize=True) * 100)

HeartDisease
0               392720
1                52255
Name: count, dtype: int64
HeartDisease
0               88.256644
1               11.743356
Name: proportion, dtype: float64


In [ ]:
X = df_prepared.drop('HeartDisease', axis=1)
y = df_prepared['HeartDisease']

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

print(f"Data latih: {X_train.shape}")
print(f"Data uji: {X_test.shape}")

Data latih: (355980, 33)
Data uji: (88995, 33)


**Config**

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

model_config = {
    'KNN': {
        'model': KNeighborsClassifier(),
        'preprocessor': distance_based_preprocessor,
        'params': {
            'model__n_neighbors': [5, 7, 9, 11],
            'model__weights': ['uniform', 'distance'],
            'model__metric': ['minkowski', 'manhattan']
        }
    }
}

**Train best estimator**

In [ ]:
memory = joblib.Memory("/content/cachedir", verbose=1)

results = {}

for model_name, config in model_config.items():
    print(model_name)

    full_pipeline = ImbPipeline(steps=[
        ('preprocessor', config['preprocessor']),

        ('smote', SMOTE(sampling_strategy=0.5, random_state=42)),

        ('model', config['model'])
    ],
        memory=memory
    )

    random_search = RandomizedSearchCV(
        estimator=full_pipeline,
        param_distributions=config['params'],
        n_iter=10,
        cv=5,
        scoring='roc_auc',
        n_jobs=-1,
        verbose=3,
        random_state=42
    )

    random_search.fit(X_train, y_train)

    best_model_pipeline = random_search.best_estimator_
    model_path = os.path.join(GDRIVE_MODEL_PATH, f'{model_name}_best_model.joblib')
    joblib.dump(best_model_pipeline, model_path)
    print(f"Saving to Google Drive {model_name} : {model_path}")

    y_pred_proba = best_model_pipeline.predict_proba(X_test)[:, 1]
    test_score = roc_auc_score(y_test, y_pred_proba)

    results[model_name] = {
        'best_score_cv': random_search.best_score_,
        'best_params': random_search.best_params_,
        'test_score_roc_auc': test_score
    }
    print(f"ROC AUC on Test Set for {model_name}: {test_score:.4f}\n")

print("\nSummary")
results_df = pd.DataFrame(results).T.sort_values(by='test_score_roc_auc', ascending=False)
print(results_df)

KNN
Fitting 5 folds for each of 10 candidates, totalling 50 fits
